### General Information

In this notebook, we are going to calculate the counter rate for each potential lineup for the recommender model.

###  Change to your path

In [3]:
import os
os.chdir(r"C:\Users\chena\Desktop\dota-2-matches\code\all_csv_files")

In [5]:
import pandas as pd
lineup = pd.read_json(r'match_lineup.json')
matches = pd.read_csv('match.csv')

### 1: Combination of the hero pool

In [6]:
heroes= pd.read_csv('heroes.csv')
import itertools
x=itertools.combinations(heroes['Hero Name'], 2)
dict1={}
for i in x:
    dict1[i]={"Win":0, "Total":0}

In [7]:
lineup1=lineup.transpose()
lineup1.head()

,radiant,dire
0,"[Rubick, Clockwerk, Treant Protector, Shadow F...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
1,"[Earthshaker, Meepo, Spirit Breaker, Queen of ...","[Alchemist, Zeus, Crystal Maiden, Spectre, Emb..."
2,"[Clockwerk, Terrorblade, Mirana, Faceless Void...","[Beastmaster, Earthshaker, Morphling, Phantom ..."
3,"[Dazzle, Phantom Assassin, Riki, Lion, Queen o...","[Brewmaster, Tiny, Lich, Venomancer, Viper]"
4,"[Juggernaut, Queen of Pain, Dark Seer, Disrupt...","[Skywrath Mage, Tusk, Zeus, Spectre, Windranger]"


In [8]:
y = matches['radiant_win'].apply(lambda win: 1 if win else 0)
classes = ['Dire Win', 'Radiant Win']
X= lineup1.merge(y, left_index=True, right_index=True)

Retrieve all the matches that radiant won the match

In [9]:
radiant_win=X[X["radiant_win"]==1]
dire_win=X[X["radiant_win"]==0]

### 2. Hero Counter  Rate 

#####    Algorithm:
        loop through radiant_win:
            loop through rad_team:
                loop through dire_team:
                    if key in dict1 then win+=1, Total+=1
                    else Total+=1

In [10]:
for index, row in radiant_win.iterrows():
    #look through every match
    rad = row[0]
    dire = row[1]
    
    for i in range (len(rad)):
        for j in range (len(dire)):
            key = (rad[i],dire[j])
            if (key in dict1):
                try:
                    dict1[key]['Win']+=1
                    dict1[key]["Total"]+=1
                except:
                    pass
            else:
                try:
                    alt_key = (dire[j],rad[i])
                    dict1[alt_key]["Total"]+=1
                except:
                    pass

#####    Algorithm:
        loop through radiant_win:
            loop through rad_team:
                loop through dire_team:
                    if key in dict1 then Total+=1
                    else win+=1, Total+=1

In [11]:
for index, row in dire_win.iterrows():
    #look through every match
    rad = row[0]
    dire = row[1]
    
    for i in range (len(rad)):
        for j in range (len(dire)):
            key = (rad[i],dire[j])
            if (key in dict1):
                try:
                    dict1[key]["Total"]+=1
                except:
                    pass
            else:
                try:
                    alt_key = (dire[j],rad[i])
                    dict1[alt_key]['Win']+=1
                    dict1[alt_key]["Total"]+=1
                except:
                    pass

In [12]:
hero_pair_counter_rate=pd.DataFrame.from_dict(dict1)
new=hero_pair_counter_rate.transpose()
new.head()

Win  Total
Abaddon Alchemist    160    322
        Axe          121    186
        Beastmaster   27     47
        Brewmaster    16     28
        Bristleback   80    135

## 3. Top 10 Hero Counter Rate

In [13]:
def counter_rate(x):
    new = x.copy()
    ### Your code goes here
    new['counter_rate'] = new['Win'] / new.Total
    return new
new1=counter_rate(new)

# Select hero pair that played more than 50 matches
more_than_50_matches = new1[new1['Total']>=50]

more_than_50_matches=more_than_50_matches.sort_values(by=['counter_rate'],ascending=False,)
top10 = more_than_50_matches.head(10)
top10

,,Win,Total,counter_rate
Omniknight,Batrider,42,53,0.792453
Vengeful Spirit,Puck,54,71,0.760563
Abaddon,Morphling,38,50,0.760000
Omniknight,Shadow Demon,97,132,0.734848
Lifestealer,Tinker,47,64,0.734375
Ursa,Tinker,73,101,0.722772
Abaddon,Bloodseeker,79,110,0.718182
Spectre,Nature's Prophet,159,222,0.716216
Wraith King,Morphling,73,102,0.715686
Undying,Oracle,40,56,0.714286


### 4. Calculate Counter rate for each team and their differences 

In [14]:

matches_hero = pd.read_csv ("model2_datasets.csv")
heroes = pd.read_csv("heroes.csv")
lineup={}

for j in range(103):
    radiant=["Rubick", "Treant Protector", "Shadow Fiend", "Spectre"]
    dire=["Ember Spirit", "Abaddon", "Templar Assassin", "Earthshaker", "Alchemist"]
    matches_hero={}
    hero = heroes["Hero Name"].iloc[j]
    if hero not in radiant or hero not in dire:
        radiant.append(hero)
    matches_hero['radiant']=radiant
    matches_hero['dire']=dire
    lineup[j]=matches_hero
import json
with open('model2_lineup.json', 'w') as fp:
    json.dump(lineup, fp)

In [15]:
X = pd.read_json (r'model2_lineup.json')
X = X.transpose()
X

,radiant,dire
0,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
1,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
2,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
3,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
4,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
...,...,...
98,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
99,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
100,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."
101,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart..."


In [16]:
diff_counter=[]


for index, row in X.iterrows():
    #look through every match
    rad = row[0]
    dire = row[1]
    total_counter_rate=0
    for i in range (len(rad)):
        each_hero=0
        for j in range (len(dire)):
            key = (rad[i],dire[j])
            if (key in dict1):
                try:
                    each_hero+=(dict1[key]['Win']/dict1[key]['Total'])
                except:
                    each_hero+=0
            else:
                try:
                    alt_key = (dire[j],rad[i])
#                     print(alt_key)
#                     print(dict1[alt_key])
                    each_hero+=(dict1[alt_key]['Win']/dict1[alt_key]['Total'])
#                     print(each_hero)
                except:
                    pass
                    each_hero+=0
#                     print (f"alt_key is {alt_key}")
        avg_counter_rate=each_hero/5
#         print(avg_counter_rate)
        total_counter_rate+=avg_counter_rate
#         print('rad_counter_rate',total_counter_rate/5)
#         print('dire_counter_rate', 1 - (total_counter_rate/5))
#     print('diff', total_counter_rate/5 - (1 - (total_counter_rate/5)))
    diff = total_counter_rate/5 - (1 - (total_counter_rate/5))
#         print(diff)
    diff_counter.append(diff)


#### Adding the difference of Counter Rate in the dataframe

In [17]:
X['diff_counter']=diff_counter
X

,radiant,dire,diff_counter
0,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",-0.011442
1,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",-0.018418
2,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.026634
3,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.026439
4,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.017701
...,...,...,...
98,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.019354
99,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.028651
100,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.041723
101,"[Rubick, Treant Protector, Shadow Fiend, Spect...","[Ember Spirit, Abaddon, Templar Assassin, Eart...",0.063220


In [18]:
df = pd.read_csv("model2_datasets.csv")
df["diff_counter"] = X["diff_counter"]

df

,radiant_Abaddon,radiant_Alchemist,radiant_Ancient Apparition,radiant_Anti-Mage,radiant_Axe,radiant_Bane,radiant_Batrider,radiant_Beastmaster,radiant_Bloodseeker,radiant_Bounty Hunter,...,dire_Viper,dire_Visage,dire_Warlock,dire_Weaver,dire_Windranger,dire_Winter Wyvern,dire_Witch Doctor,dire_Wraith King,dire_Zeus,diff_counter
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.011442
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.018418
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.026634
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.026439
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.017701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000501
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.019354
99,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.028651
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.041723


In [49]:
# df.to_csv("recommender.csv")